In [4]:
# Load and read the SQL file to extract table definitions
file_path = "/Users/sangameshwaruppe/custom_app/Mavenmovies (1).sql"

# Read the SQL file content
with open(file_path, "r", encoding="utf-8") as file:
    sql_content = file.read()

# Display a snippet of the content for analysis
sql_content[:1000]  # Showing first 1000 characters to check structure


'-- Sakila Sample Database Schema\n-- Version 1.0\n\n-- Copyright (c) 2006, 2015, Oracle and/or its affiliates. \n-- All rights reserved.\n\n-- Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:\n\n--  * Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.\n--  * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.\n--  * Neither the name of Oracle nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.\n\n-- THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTAB

In [5]:
import re

# Extract CREATE TABLE statements from the SQL file
create_table_statements = re.findall(r'CREATE TABLE.*?;', sql_content, re.DOTALL)

# Display the number of tables found
len(create_table_statements)


19

In [6]:
# Extract primary keys and foreign keys from the CREATE TABLE statements

pk_dict = {}  # Dictionary to store table names and their primary keys
fk_dict = {}  # Dictionary to store table names and their foreign keys

for table_stmt in create_table_statements:
    # Extract table name
    table_name_match = re.search(r'CREATE TABLE `(.*?)`', table_stmt)
    if table_name_match:
        table_name = table_name_match.group(1)
        
        # Extract primary key
        pk_match = re.search(r'PRIMARY KEY \((.*?)\)', table_stmt)
        if pk_match:
            pk_columns = pk_match.group(1).replace('`', '').split(', ')
            pk_dict[table_name] = pk_columns
        
        # Extract foreign keys
        fk_matches = re.findall(r'FOREIGN KEY \((.*?)\) REFERENCES `(.*?)` \((.*?)\)', table_stmt)
        if fk_matches:
            fk_dict[table_name] = [(fk[0].replace('`', ''), fk[1], fk[2].replace('`', '')) for fk in fk_matches]

# Display extracted PKs and FKs
pk_dict, fk_dict


({}, {})

In [7]:
# Refined extraction approach for PKs and FKs
pk_dict = {}
fk_dict = {}

for table_stmt in create_table_statements:
    # Extract table name
    table_name_match = re.search(r'CREATE TABLE\s+`?(\w+)`?', table_stmt, re.IGNORECASE)
    if table_name_match:
        table_name = table_name_match.group(1)

        # Extract primary key
        pk_match = re.search(r'PRIMARY KEY\s*\(([^)]+)\)', table_stmt, re.IGNORECASE)
        if pk_match:
            pk_columns = [col.strip('` ') for col in pk_match.group(1).split(',')]
            pk_dict[table_name] = pk_columns

        # Extract foreign keys
        fk_matches = re.findall(r'FOREIGN KEY\s*\(([^)]+)\)\s*REFERENCES\s*`?(\w+)`?\s*\(([^)]+)\)', table_stmt, re.IGNORECASE)
        if fk_matches:
            fk_dict[table_name] = [(fk[0].strip('` '), fk[1], fk[2].strip('` ')) for fk in fk_matches]

# Display extracted PKs and FKs again
pk_dict, fk_dict


({'actor': ['actor_id'],
  'actor_award': ['actor_award_id'],
  'address': ['address_id'],
  'advisor': ['advisor_id'],
  'category': ['category_id'],
  'city': ['city_id'],
  'country': ['country_id'],
  'customer': ['customer_id'],
  'film': ['film_id'],
  'film_actor': ['actor_id', 'film_id'],
  'film_category': ['film_id', 'category_id'],
  'film_text': ['film_id'],
  'inventory': ['inventory_id'],
  'investor': ['investor_id'],
  'language': ['language_id'],
  'payment': ['payment_id'],
  'rental': ['rental_id'],
  'staff': ['staff_id'],
  'store': ['store_id']},
 {'address': [('city_id', 'city', 'city_id')],
  'city': [('country_id', 'country', 'country_id')],
  'customer': [('address_id', 'address', 'address_id'),
   ('store_id', 'store', 'store_id')],
  'film': [('language_id', 'language', 'language_id'),
   ('original_language_id', 'language', 'language_id')],
  'film_actor': [('actor_id', 'actor', 'actor_id'),
   ('film_id', 'film', 'film_id')],
  'film_category': [('film_id'